In [ ]:
import torch
import torchvision

import matplotlib.pyplot as plt
import torch
import torchvision

import requests
from tqdm import tqdm

from torch import nn
from torchvision import transforms

from pathlib import Path

try:
  from torchinfo import summary
except:
  print("[INFO] Couldn't find torchinfo... installing it.")
  !pip install -q torchinfo
  from torchinfo import summary


try:
  from modular import download_image_data, data_setup, train_model, save
except:
  # Get the going_modular scripts
  print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
  !git clone https://github.com/AaronJ59/Projects.git
  !mv Projects/BiteID/modular/modular modular
  !mv Projects/BiteID/models models
  from modular import download_image_data, data_setup, train_model, save
  !rm -rf Projects


[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'Projects'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 163 (delta 33), reused 10 (delta 1), pack-reused 54 (from 2)
Receiving objects: 100% (163/163), 109.87 MiB | 36.80 MiB/s, done.
Resolving deltas: 100% (42/42), done.
mv: cannot stat 'Projects/BiteID/models': No such file or directory


In [ ]:
import os
import zipfile

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
os.makedirs("data/food80_binary")

with zipfile.ZipFile("food80_binary.zip", "r") as zip_ref:
  zip_ref.extractall("data/food80_binary")

In [ ]:
def create_effnetb2_model(seed:int=42):


  # Import efficientnetb2

  # Get the weights, transforms of the model and make the model
  efficientnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  effnetb2_transform = efficientnetb2_weights.transforms()
  effnetb2 = torchvision.models.efficientnet_b2(weights=efficientnetb2_weights)

  # Freeze the base layers of efficientnetb2
  for param in effnetb2.features.parameters():
    param.requires_grad = False



  # Change classifier head of effnetb2 to suit binary classification
  torch.manual_seed(seed)
  effnetb2.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=1), # 1 out_feature because I'm making a binary feature extractor
  )

  return effnetb2, effnetb2_transform




# Create an instance
effnetb2, effnetb2_transform = create_effnetb2_model(seed=42)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 71.7MB/s]


In [ ]:
data_dir = Path("data")
train_dir = data_dir / "food80_binary" / "train"
test_dir = data_dir / "food80_binary" / "test"

NUM_WORKERS = os.cpu_count()

In [ ]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=effnetb2_transform,
                                                                               num_workers=NUM_WORKERS,
                                                                               batch_size=32)

In [ ]:
optimizer = torch.optim.Adam(params=effnetb2.parameters(),
                             lr=.001)

loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.dataloader,
               optimizer: torch.optim.Optimizer,
               loss_fn:torch.nn.Module,
               device: torch.device):

  model.train()

  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.float().to(device)

    # Do the forward pass
    y_pred_logits = model(X).squeeze(dim=1)

    # Calculate the loss
    loss = loss_fn(y_pred_logits, y)
    train_loss = train_loss + loss.item()


    # Optimize graidnet
    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    # prediction logits to hard 0 or 1 predictions by thresholding at 0.0
    y_pred_labels = (y_pred_logits >= 0).long()
    # Calculate the accuracy for the current batch by dividing the number of correct predictions
    # by the batch size, and add it to the running total accuracy.
    train_acc = train_acc + ((y_pred_labels == y.long()).sum().item() / len(y_pred_labels))

  # Get average accuracy and loss per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)

  return train_loss, train_acc



In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device):
  model.eval()

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.float().to(device)

      # Do the forward pass
      test_pred_logits = model(X).squeeze(dim=1)


      # Calculate the loss
      loss = loss_fn(test_pred_logits, y)
      test_loss = test_loss + loss.item()

      # prediction logits to prediction label
      test_pred_labels = (test_pred_logits >= 0).long()

      test_acc = test_acc + ((test_pred_labels == y.long()).sum().item() / len(test_pred_labels))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)

    return test_loss, test_acc






In [ ]:
from tqdm import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          device: device,
          epochs: int):

  results = {
      "train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  model.to(device)
  print(f"Training model for {epochs} epochs")
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       optimizer=optimizer,
                                       loss_fn=loss_fn,
                                       device=device)

    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)


    print(f"\nEpoch: {epoch} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | test_loss: {test_loss:.4f} | test_acc: {test_acc:.4f}")



    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results

In [ ]:
effnetb2_binary_results = train(model=effnetb2,
                                train_dataloader=train_dataloader,
                                test_dataloader=test_dataloader,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                device=device,
                                epochs=10)

Training model for 10 epochs


 10%|█         | 1/10 [00:30<04:37, 30.86s/it]


Epoch: 0 | train_loss: 0.4001 | train_acc: 0.8466 | test_loss: 0.2441 | test_acc: 0.9697


 20%|██        | 2/10 [00:53<03:26, 25.85s/it]


Epoch: 1 | train_loss: 0.1842 | train_acc: 0.9754 | test_loss: 0.1592 | test_acc: 0.9892


...


Epoch: 7 | train_loss: 0.0619 | train_acc: 0.9868 | test_loss: 0.0612 | test_acc: 0.9931


 90%|█████████ | 9/10 [03:28<00:22, 22.40s/it]


Epoch: 8 | train_loss: 0.0611 | train_acc: 0.9859 | test_loss: 0.0583 | test_acc: 0.9931


100%|██████████| 10/10 [03:49<00:00, 22.91s/it]


Epoch: 9 | train_loss: 0.0577 | train_acc: 0.9872 | test_loss: 0.0519 | test_acc: 0.9918


In [ ]:
save.save_model(model=effnetb2, target_dir="models", model_name="effnetb2_binary_food80_model.pth")

Saving model to: models/effnetb2_binary_food80_model.pth
